https://www.edarabia.com/schools/sweden/

scrape this for school list

In [ ]:
# get drivers ready for selenium scraping
from zipfile import ZipFile
import wget

wget.download('https://github.com/mozilla/geckodriver/releases/download/v0.27.0/geckodriver-v0.27.0-win64.zip', 'gecko.zip')
with ZipFile('gecko.zip', 'r') as z: z.extractall('.')

In [2]:
# scrape https://www.edarabia.com/schools/sweden/ for top schools of sweden
from selenium import webdriver

driver = webdriver.Firefox(executable_path='geckodriver.exe')
driver.get('https://www.edarabia.com/schools/sweden/')

In [12]:
# Upon inspection, school names are part of a hyperlink under <h5> tags
links = driver.find_elements_by_xpath('//h5/a')
for i, link in enumerate(links):
    print(i, link.text)

0 
1 
2 
3 
4 Bladins International School
5 Deutsche Schule Stockholm
6 Engelska Skolan Norr AB
7 Fryshuset School
8 International IT College of Sweden
9 Lycee Francais Saint Louis in Stockholm
10 ProCivitas Private Gymnasium Stockholm
11 Swedish Finnish School
12 The Tanto International School
13 Thea Private Grundskola


In [14]:
# there seem to be empty links, lets remove that
schools = [link.text for link in links if link.text]
schools

['Bladins International School',
 'Deutsche Schule Stockholm',
 'Engelska Skolan Norr AB',
 'Fryshuset School',
 'International IT College of Sweden',
 'Lycee Francais Saint Louis in Stockholm',
 'ProCivitas Private Gymnasium Stockholm',
 'Swedish Finnish School',
 'The Tanto International School',
 'Thea Private Grundskola']